Este notebook recorre los archivos html de la carpeta `.data`, y exporta de ahi el js del cuatrimestre

Ese `horarios.js` hay que ponerlo en la carpeta src/data

La carpeta data tiene que tener un archivo html por **cada carrera**

El html se consigue de la siguiente manera
- Loggearse al SIU, ir a "Reportes" > "Oferta de comisiones"
- Conseguir el HTML de ese sitio (CTRL+U)
- Ese HTML abrirlo en un editor de texto y buscar el texto `kernel.renderer.on_arrival`
- Esa funcion recibe un objeto. Ese objeto tiene una clave `content`
- El **valor** de content es el html que va en la carpeta `.data`

In [1]:
import os

replacements = {
    r'\"': r'"',
    r'<\/': r'</',
    r'\n': '',
    '\n': '',
    '\t': '',
    r'\t': '',
    'á':'a',
    'é':'e',
    'í':'i',
    'ó':'o',
    'ú':'u',
    'Á':'A',
    'É':'E',
    'Í':'I',
    'Ó':'O',
    'Ú':'U'
}


for f in os.listdir():
    if '.html' not in f:
        continue
    
    with open(f, 'r+') as fsub:
        txt = fsub.read()
        txt = txt.encode('iso-8859-1').decode('unicode_escape')
        for k,v in replacements.items():
            txt = txt.replace(k,v)
        fsub.seek(0)
        fsub.write(txt)
        fsub.truncate()

/home/delmazo/.local/lib/python3.7/site-packages/ipykernel_launcher.py:29: DeprecationWarning: invalid escape sequence '\/'


In [2]:
from bs4 import BeautifulSoup
from collections import namedtuple
import re
import json

REGEX = r'(.*) (\(\d\d\d\d\))'
materia_regex = re.compile(REGEX)
DIAS_DE_LA_HERMOSA_SEMANA = ["Domingo", "Lunes", "Martes", "Miercoles", "Jueves", "Viernes", "Sabado"]

In [3]:
def get_ids_materias(soup):
    codigos_materias = []
    actividades = soup.find_all('div', {'class': 'js-recuadro_actividad'})

    for act in actividades:
        nombre, codigo = materia_regex.search(act.get('actividad')).groups()
        codigo = re.sub('\(|\)', "", codigo)
        codigos_materias.append(codigo)
    return codigos_materias

soup = BeautifulSoup(open("sistemas.html"), "html.parser")
get_ids_materias(soup)

['9142',
 '9524',
 '8102',
 '9514',
 '9515',
 '9502',
 '9135',
 '8101',
 '9504',
 '9564',
 '9505',
 '8636',
 '9568',
 '9559',
 '9141',
 '9530',
 '9139',
 '9535',
 '9802',
 '9803',
 '9801',
 '9804',
 '9805',
 '9525',
 '9140',
 '9548',
 '9124',
 '8111',
 '9520',
 '9521',
 '9104',
 '9105',
 '9106',
 '9558',
 '9557',
 '8104',
 '9560',
 '9519',
 '9503',
 '9508',
 '9509',
 '9506',
 '9507']

In [4]:
def get_ids_comisiones(soup):
    ids_comisiones = []
    comisiones_soup = soup.find_all('table', {'class': 'comision'})
    for comision in comisiones_soup:
        comision_id = comision.get('comision')
        ids_comisiones.append(comision_id)
    return ids_comisiones

def get_materias(soup):
    materias = []
    actividades = soup.find_all('div', {'class': 'js-recuadro_actividad'})

    for act in actividades:
        nombre, codigo = materia_regex.search(act.get('actividad')).groups()
        codigo = re.sub('\(|\)', "", codigo)
        ids_comisiones = get_ids_comisiones(act)
        
        materias.append({
            "codigo": codigo,
            "nombre": nombre,
            "cursos": ids_comisiones
        })
        
    return materias

soup = BeautifulSoup(open("informatica.html"), "html.parser")
get_materias(soup)

[{'codigo': '7544',
  'nombre': 'ADM. Y CONTROL DE PROY. INFORMATICOS I',
  'cursos': ['29543']},
 {'codigo': '7546',
  'nombre': 'ADM. Y CONTROL DE PROY. INFORMATICOS II',
  'cursos': ['29545']},
 {'codigo': '6108',
  'nombre': 'ALGEBRA II A',
  'cursos': ['29579',
   '29581',
   '29582',
   '29583',
   '29584',
   '29585',
   '29586',
   '29587',
   '29588',
   '29589',
   '29590',
   '29591',
   '29592',
   '29593',
   '29594',
   '29595',
   '29596',
   '29597',
   '29598',
   '31529',
   '31530',
   '31531']},
 {'codigo': '7540',
  'nombre': 'ALGORITMOS Y PROGRAMACION I',
  'cursos': ['29606', '29608', '29609', '29610']},
 {'codigo': '7541',
  'nombre': 'ALGORITMOS Y PROGRAMACION II',
  'cursos': ['29623', '29624', '29625', '29626']},
 {'codigo': '7507',
  'nombre': 'ALGORITMOS Y PROGRAMACION III',
  'cursos': ['29635', '29636', '31646']},
 {'codigo': '6606',
  'nombre': 'ANALISIS DE CIRCUITOS',
  'cursos': ['31551', '31556', '31564']},
 {'codigo': '7509',
  'nombre': 'ANALISIS DE

In [5]:
def get_comision(soup, codigo):
    comision_soup = soup.find('table', {'comision': codigo})
    docentes = comision_soup.get('docentes')
    comision_id = comision_soup.get('comision')
    clases = []

    for dia_semana_soup in comision_soup.find_all('tr', {'class': ['js-dia']}):
        dia_semana = dia_semana_soup.get('dia_sem')
        horas_soup = dia_semana_soup.find('td', text=re.compile(r'\d\d\:\d\d'))
        if not horas_soup:
            continue
        horas = horas_soup.contents[0]
        inicio, fin = horas.split(' a ')

        clases.append({'dia': DIAS_DE_LA_HERMOSA_SEMANA.index(dia_semana), 'inicio': inicio, 'fin': fin})

    return {
        "docentes": docentes,
        "codigo": codigo,
        "clases": clases
    }

soup = BeautifulSoup(open("sistemas.html"), "html.parser")
get_comision(soup, "30851")

{'docentes': 'COLOMBO PABLO MARTIN, RAMONET JUAN ANTONIO, RAMOS SILVIA ADRIANA, MARIN EMILIO GERMAN, TERZANO IVAN GUILLERMO',
 'codigo': '30851',
 'clases': [{'dia': 3, 'inicio': '18:30', 'fin': '21:30'},
  {'dia': 4, 'inicio': '09:00', 'fin': '11:30'},
  {'dia': 6, 'inicio': '10:00', 'fin': '13:00'}]}

In [6]:
NOMBRES = {
    "informatica.html": "Ingeniería en Informática",
    "civil.html": "Ingeniería Civil",
    "industrial.html": "Ingeniería Industrial",
    "mecanica.html": "Ingeniería Mecánica",
    "quimica.html": "Ingeniería Química",
    "sistemas.html": "Licenciatura en Análisis de Sistemas",
    "sistemas86.html": "Licenciatura en Análisis de Sistemas (Plan viejo)",
    "electronica.html": "Ingeniería Electrónica",
    "alimentos.html": "Ingeniería de Alimentos",
    "agrimensura.html": "Ingeniería en Agrimensura",
    "naval-mecanica.html": "Ingeniería Naval y Mecánica",
    "petroleo.html": "Ingeniería en Petroleo",
    "electricista.html": "Ingeniería Electricista"
}

carreras = []
codigos_materias_agregadas = []
materias = []
codigos_comisiones_agregadas = []
comisiones = []

for f in os.listdir():
    if '.html' not in f:
        continue
        
    soup = BeautifulSoup(open(f), "html.parser")

    carreras.append({
        "nombre": NOMBRES[f],
        'materias': get_ids_materias(soup)
    })
    
    materias_carrera = get_materias(soup)
    materias_carrera = [x for x in materias_carrera if x['codigo'] not in codigos_materias_agregadas]
    for m in materias_carrera:
        codigos_materias_agregadas.append(m['codigo'])
        comisiones_materia = m.get('cursos')
        comisiones_materia = [x for x in comisiones_materia if x not in codigos_comisiones_agregadas]
        for c in comisiones_materia:
            codigos_comisiones_agregadas.append(c)
            comisiones.append(get_comision(soup, c))
            
        comisiones_sin_clases = filter(lambda x: not x['clases'], comisiones)
        comisiones_sin_clases_codigos = map(lambda x: x['codigo'], comisiones_sin_clases)
        m['cursos'] = list(filter(lambda x: x not in comisiones_sin_clases_codigos, m['cursos']))
    
    comisiones = list(filter(lambda x: x['clases'], comisiones))
    materias.extend(materias_carrera) 
    
obj = {
    "cuatrimestre": "2020C2",
    "carreras": carreras,
    "materias": materias,
    "cursos": comisiones
}

obj_dump = json.dumps(obj, indent=2, ensure_ascii=False)        

with open('horarios.js', 'w') as fw:
    fw.write("export const data = ")
    fw.write(obj_dump)

print(obj_dump)    

{
  "cuatrimestre": "2020C2",
  "carreras": [
    {
      "nombre": "Ingeniería Mecánica",
      "materias": [
        "6108",
        "6416",
        "6103",
        "6113",
        "7512",
        "6741",
        "6730",
        "7501",
        "6713",
        "6714",
        "6742",
        "6723",
        "6707",
        "6603",
        "6504",
        "6725",
        "6762",
        "6716",
        "6411",
        "6412",
        "6413",
        "6218",
        "6201",
        "6203",
        "7701",
        "7802",
        "7803",
        "7801",
        "7804",
        "7805",
        "7101",
        "7125",
        "6719",
        "6506",
        "6750",
        "6718",
        "6206",
        "6207",
        "6711",
        "6721",
        "6702",
        "6414",
        "6726",
        "6744",
        "6106",
        "6724",
        "6734",
        "6729",
        "7705",
        "6301",
        "6743",
        "6722",
        "6728",
        "6717",
        "6756",
        "